In [38]:
import torch

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [42]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [44]:
string_to_text=  {ch:i for i , ch in enumerate(chars)}
# Output: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ..., 'z': 25}
int_to_string = {i:ch for i, ch in enumerate(chars)}
# Output: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', ..., 25: 'z'}

encode = lambda s: [string_to_text[c] for c in s]
# Output: [7, 4, 11, 11, 14]

decode = lambda l: ''.join([int_to_string[i] for i in l])
# Output: "hello"

In [46]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([71, 22, 53, 59, 57, 54, 43, 60, 42, 59, 48, 54, 53,  0,  0,  0, 19, 54,
        51, 50, 51, 54, 57, 44,  5,  1, 51, 44, 46, 44, 53, 43, 58,  5,  1, 52,
        64, 59, 47, 58,  1, 40, 53, 43,  1, 45, 40, 48, 57, 64,  1, 59, 40, 51,
        44, 58,  1, 47, 40, 61, 44,  1, 45, 54, 51, 51, 54, 62, 44, 43,  1, 42,
        47, 48, 51, 43, 47, 54, 54, 43,  0, 59, 47, 57, 54, 60, 46, 47,  1, 59,
        47, 44,  1, 40, 46, 44, 58,  5,  1, 45])


In [48]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [28]:
block_size = 8
# how a language model might be trained to predict the next token given a context.
x = train_data[:block_size]
y=  train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([71]) target is tensor(22)
when input is tensor([71, 22]) target is tensor(53)
when input is tensor([71, 22, 53]) target is tensor(59)
when input is tensor([71, 22, 53, 59]) target is tensor(57)
when input is tensor([71, 22, 53, 59, 57]) target is tensor(54)
when input is tensor([71, 22, 53, 59, 57, 54]) target is tensor(43)
when input is tensor([71, 22, 53, 59, 57, 54, 43]) target is tensor(60)
when input is tensor([71, 22, 53, 59, 57, 54, 43, 60]) target is tensor(42)
